## Parsing CSV and Excel files
 

In [1]:
import pandas as pd
import os

In [2]:
os.makedirs("data/unstructured_files", exist_ok=True)

In [3]:
data = {
    'Product': ['Laptop', 'Mouse', 'Keyboard', 'Monitor', 'Webcam'],
    'Category': ['Electronics', 'Accessories', 'Accessories', 'Electronics', 'Electronics'],
    'Price': [999.99, 29.99, 79.99, 299.99, 89.99],
    'Stock': [50, 200, 150, 75, 100],
    'Description': [
        'High-performance laptop with 16GB RAM and 512GB SSD',
        'Wireless optical mouse with ergonomic design',
        'Mechanical keyboard with RGB backlighting',
        '27-inch 4K monitor with HDR support',
        '1080p webcam with noise cancellation'
    ]
}

In [4]:
df = pd.DataFrame(data)
df.to_csv('data/unstructured_files/products.csv', index=False)

In [5]:
with pd.ExcelWriter('data/unstructured_files/products.xlsx') as writer:
    df.to_excel(writer, sheet_name='Products', index=False)

    summary_data = {
        'Category':['Electronics', 'Accessories'],
        'Total_Items': [3,2],
        'Total_Value': [1389.97, 109.98]
    }
    pd.DataFrame(summary_data).to_excel(writer, sheet_name='Summary', index = False)

## CSV Processing

In [6]:
from langchain_community.document_loaders import CSVLoader, UnstructuredCSVLoader

In [7]:
# Method 1- row based Documents
print("CSV Loader-Row Based Documents")
csv_loader = CSVLoader(
    file_path='data/unstructured_files/products.csv',
    encoding='utf-8',
    csv_args={
        'delimiter':",",
        'quotechar':'"',
    }
)

csv_docs = csv_loader.load()
print(F"Loaded {len(csv_docs)} documents")
for i,doc in enumerate(csv_docs):
    print(f"Document: {i+1}")
    print(f"Content: {doc.page_content}")
    print(f"Metadata: {doc.metadata}")
    print("\n")

CSV Loader-Row Based Documents
Loaded 5 documents
Document: 1
Content: Product: Laptop
Category: Electronics
Price: 999.99
Stock: 50
Description: High-performance laptop with 16GB RAM and 512GB SSD
Metadata: {'source': 'data/unstructured_files/products.csv', 'row': 0}


Document: 2
Content: Product: Mouse
Category: Accessories
Price: 29.99
Stock: 200
Description: Wireless optical mouse with ergonomic design
Metadata: {'source': 'data/unstructured_files/products.csv', 'row': 1}


Document: 3
Content: Product: Keyboard
Category: Accessories
Price: 79.99
Stock: 150
Description: Mechanical keyboard with RGB backlighting
Metadata: {'source': 'data/unstructured_files/products.csv', 'row': 2}


Document: 4
Content: Product: Monitor
Category: Electronics
Price: 299.99
Stock: 75
Description: 27-inch 4K monitor with HDR support
Metadata: {'source': 'data/unstructured_files/products.csv', 'row': 3}


Document: 5
Content: Product: Webcam
Category: Electronics
Price: 89.99
Stock: 100
Description: 1

In [8]:
from typing import List
from langchain_core.documents import Document
#Method 2: Custom CSV processing for better control

def intelligent_csv_processor(filepath: str) -> List[Document]:
    """Process CSV with intelligent document creation"""
    df = pd.read_csv(filepath)
    documents = []


    # Setrategy 1: one document per row with structured content
    for idx, row in df.iterrows():
        #Create a structured document
        content = f"""Product Information:
        Name:{row['Product']}
        Category:{row['Category']}
        Price:${row['Price']}
        Stock:{row['Stock']} units
        Description:{row['Description']}"""

        doc = Document(
            page_content=content,
            matadata = {
                'source': filepath,
                'row_index':idx,
                'product_name':row['Product'],
                'category':row['Category'],
                'price':row['Price'],
                'data_type':'product_info'
            }
        )

        documents.append(doc)
    return documents


In [9]:
docs = intelligent_csv_processor("data/unstructured_files/products.csv")

In [10]:
docs

[Document(metadata={}, page_content='Product Information:\n        Name:Laptop\n        Category:Electronics\n        Price:$999.99\n        Stock:50 units\n        Description:High-performance laptop with 16GB RAM and 512GB SSD'),
 Document(metadata={}, page_content='Product Information:\n        Name:Mouse\n        Category:Accessories\n        Price:$29.99\n        Stock:200 units\n        Description:Wireless optical mouse with ergonomic design'),
 Document(metadata={}, page_content='Product Information:\n        Name:Keyboard\n        Category:Accessories\n        Price:$79.99\n        Stock:150 units\n        Description:Mechanical keyboard with RGB backlighting'),
 Document(metadata={}, page_content='Product Information:\n        Name:Monitor\n        Category:Electronics\n        Price:$299.99\n        Stock:75 units\n        Description:27-inch 4K monitor with HDR support'),
 Document(metadata={}, page_content='Product Information:\n        Name:Webcam\n        Category:Electr

### Excel Processing

In [11]:
# Strategy 1: using excel for full control

def process_excel_with_pandas(filepath: str)-> List[Document]:
    """Process Excel with sheet awarness"""
    documents = []

    # Read all sheets
    excel_file = pd.ExcelFile(filepath)
    for sheet_name in  excel_file.sheet_names :
        df = pd.read_excel(filepath, sheet_name = sheet_name)

        #create document for each sheet

        sheet_content = f"sheet: {sheet_name}"
        sheet_content += f"columns: {','.join(df.columns)}"
        sheet_content += f"rows: {len(df)} \n \n"
        sheet_content += df.to_string(index=False)

        document = Document(
            page_content = sheet_content,
            metadata = {
                'source': filepath,
                'Sheet_name': sheet_name,
                'num_rows':len(df),
                'num_columns': len(df.columns),
                'data_type':'product_info'
            }
        )
        documents.append(document)
    return documents

In [12]:
excel_docs = process_excel_with_pandas('data/unstructured_files/products.xlsx')


In [13]:
print(f"Processed {len(excel_docs)} sheets")

Processed 2 sheets


In [14]:
excel_docs

[Document(metadata={'source': 'data/unstructured_files/products.xlsx', 'Sheet_name': 'Products', 'num_rows': 5, 'num_columns': 5, 'data_type': 'product_info'}, page_content='sheet: Productscolumns: Product,Category,Price,Stock,Descriptionrows: 5 \n \n Product    Category  Price  Stock                                         Description\n  Laptop Electronics 999.99     50 High-performance laptop with 16GB RAM and 512GB SSD\n   Mouse Accessories  29.99    200        Wireless optical mouse with ergonomic design\nKeyboard Accessories  79.99    150           Mechanical keyboard with RGB backlighting\n Monitor Electronics 299.99     75                 27-inch 4K monitor with HDR support\n  Webcam Electronics  89.99    100                1080p webcam with noise cancellation'),
 Document(metadata={'source': 'data/unstructured_files/products.xlsx', 'Sheet_name': 'Summary', 'num_rows': 2, 'num_columns': 3, 'data_type': 'product_info'}, page_content='sheet: Summarycolumns: Category,Total_Items,To